In [1]:
import os
import mlflow
import pickle
import pandas as pd
import uuid

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

In [13]:
year = 2021
month = 3
taxi_type = "green"

if not os.path.exists(f"output/{taxi_type}"): 
    os.makedirs(f"output/{taxi_type}") 

input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"  # download from link
output_file = f"output/{taxi_type}/{year:04d}-{month:02d}.parquet"

EXPERIMENT_ID = 1
RUN_ID = os.getenv("RUN_ID", "553def03f5224f649fe56bc1567daccc")
logged_model = f"gs://pytholic-mlops-zoomcamp-artifacts/{EXPERIMENT_ID}/{RUN_ID}/artifacts/model"
service_account_key_path = os.path.expanduser('~') + "/service_account_key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key_path

In [14]:
def gen_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]    

    df["ride_id"] = gen_uuids(len(df))
    return df

def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [15]:
def load_model(experiment_id, run_id):
    logged_model = f"gs://pytholic-mlops-zoomcamp-artifacts/{experiment_id}/{run_id}/artifacts/model"
    model = mlflow.pyfunc.load_model(logged_model)
    return model
    

def apply_model(input_file, experiment_id, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    model = load_model(experiment_id, run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result["ride_id"] = df["ride_id"]
    df_result["lpep_pickup_datetime"] = df["lpep_pickup_datetime"]
    df_result["PULocationID"] = df["PULocationID"]
    df_result["DOLocationID"] = df["DOLocationID"]
    df_result["actual_duration"] = df["duration"]
    df_result["predicted_duration"] = y_pred
    df_result["diff"] = df_result["actual_duration"] - df_result["predicted_duration"]
    df_result["model_version"] = RUN_ID

    df_result.to_parquet(output_file, index=False)

    

In [16]:
apply_model(input_file, EXPERIMENT_ID, RUN_ID, output_file)

2024/01/16 08:57:22 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - psutil (current: 5.9.0, required: psutil==5.9.7)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [17]:
!ls output/green

2021-02.parquet  2021-03.parquet
